In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 15.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

Data preprocesseing

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


normalizing images (from rgb to grey scale)

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

reshaping

In [8]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

build the model

In [9]:
def build_model(hp) :
  model = keras.Sequential([
     keras.layers.Conv2D(
          filters= hp.Int('conv_1_filter', min_value = 32, max_value= 128, step = 16),
          kernel_size= hp.Choice('conv_1_kernel', values = [3,5]),
          activation= 'relu',
          input_shape= (28,28,1)
      ),
       keras.layers.Conv2D(
          filters= hp.Int('conv_2_filter', min_value = 32, max_value= 64, step = 16),
          kernel_size= hp.Choice('conv_2_kernel', values = [3,5]),
          activation= 'relu'
       ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units= hp.Int('dense_1_units', min_value = 32, max_value= 128, step = 16),
          activation ='relu'
      ),
      keras.layers.Dense(10, activation ='softmax')
  ])

  model.compile(optimizer= keras.optimizers.Adam(hp.Choice('learning rate', values= [1e-2, 1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics =['accuracy'])
  return model


In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='fashion fashion')

In [12]:
tuner_search.search(train_images,train_labels, epochs=3, validation_split= 0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8820000290870667

Best val_accuracy So Far: 0.9153333306312561
Total elapsed time: 00h 03m 45s


In [13]:
model = tuner_search.get_best_models(num_models= 1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        36896     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 80)                1474640   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 1,513,626
Trainable params: 1,513,626
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs= 10, validation_split= 0.1, initial_epoch= 3)

Epoch 4/10
1688/1688 [==============================] - 11s 5ms/step - loss: 0.1464 - accuracy: 0.9457 - val_loss: 0.2469 - val_accuracy: 0.9148
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1079 - accuracy: 0.9597 - val_loss: 0.2982 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0800 - accuracy: 0.9710 - val_loss: 0.3286 - val_accuracy: 0.9128
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0582 - accuracy: 0.9780 - val_loss: 0.3664 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0446 - accuracy: 0.9840 - val_loss: 0.4040 - val_accuracy: 0.9045
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0349 - accuracy: 0.9883 - val_loss: 0.4452 - val_accuracy: 0.9108
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0281 - accuracy: 0.9904 - val_loss: 0.5456 - val_accura